# Mathematical Optimisation - Project
This project aims at ricreating the results reported in:

[A fix-and-optimize heuristic for the Unrelated Parallel Machine Scheduling Problem
](https://www.sciencedirect.com/science/article/pii/S0305054823003684)

In [1]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output

r.seed(10)
  
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

## Creation of the matrices
- P stores the execution times
- S stores the setup times
The matrices are implemented using dictionaries to handle the indexes not starting from 0

### Small instance presented in the paper

In [2]:
P_dict = {
            (1,1): 2,(1,2): 2,(1,3): 3,(1,4): 4,(2,1): 3,(2,2): 3,(2,3): 5,(2,4): 3,
         }

S_dict = {
            (1,1,1): 0,(1,1,2): 2,(1,1,3): 1,(1,1,4): 1,(1,2,1): 2,(1,2,2): 0,(1,2,3): 1,(1,2,4): 2,
            (1,3,1): 3,(1,3,2): 1,(1,3,3): 0,(1,3,4): 3,(1,4,1): 2,(1,4,2): 1,(1,4,3): 2,(1,4,4): 0,
            (2,1,1): 0,(2,1,2): 3,(2,1,3): 2,(2,1,4): 1,(2,2,1): 2,(2,2,2): 0,(2,2,3): 3,(2,2,4): 2,
            (2,3,1): 3,(2,3,2): 2,(2,3,3): 0,(2,3,4): 3,(2,4,1): 2,(2,4,2): 2,(2,4,3): 3,(2,4,4): 0,
         }

### Instance generator
The parameters are:
- The cardinality of the set of jobs N
- The cardinality of the set of machines M
- The minimum and the maximum execution time
- The minimum and the maximum setup time

In [3]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 10,
                      min_setup_time = 1,max_setup_time = 3):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 50
M_cardinality = 2

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality,
                                   min_execution_time = 1, max_execution_time = 100,
                                   min_setup_time = 1, max_setup_time = 100)

# P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality)

In [4]:
print(P_dict)
print(S_dict)

{(1, 1): 74, (1, 2): 5, (1, 3): 55, (1, 4): 62, (1, 5): 74, (1, 6): 2, (1, 7): 27, (1, 8): 60, (1, 9): 63, (1, 10): 36, (1, 11): 84, (1, 12): 21, (1, 13): 5, (1, 14): 67, (1, 15): 63, (1, 16): 42, (1, 17): 10, (1, 18): 32, (1, 19): 96, (1, 20): 47, (1, 21): 6, (1, 22): 54, (1, 23): 18, (1, 24): 78, (1, 25): 46, (1, 26): 49, (1, 27): 54, (1, 28): 37, (1, 29): 87, (1, 30): 34, (1, 31): 59, (1, 32): 23, (1, 33): 88, (1, 34): 39, (1, 35): 85, (1, 36): 47, (1, 37): 18, (1, 38): 59, (1, 39): 99, (1, 40): 31, (1, 41): 57, (1, 42): 79, (1, 43): 49, (1, 44): 6, (1, 45): 75, (1, 46): 1, (1, 47): 31, (1, 48): 18, (1, 49): 25, (1, 50): 39, (2, 1): 69, (2, 2): 47, (2, 3): 99, (2, 4): 31, (2, 5): 41, (2, 6): 86, (2, 7): 71, (2, 8): 58, (2, 9): 56, (2, 10): 61, (2, 11): 9, (2, 12): 84, (2, 13): 75, (2, 14): 42, (2, 15): 65, (2, 16): 21, (2, 17): 29, (2, 18): 53, (2, 19): 31, (2, 20): 5, (2, 21): 5, (2, 22): 64, (2, 23): 39, (2, 24): 78, (2, 25): 85, (2, 26): 10, (2, 27): 69, (2, 28): 11, (2, 29): 20,

The sets N,M,N0 are created again to print the results down the line

In [5]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

# Exact Solution

The problem is solved exactly using the Solver class.

Then, the results are printed. Note that job 0 is a dummy job used to represent the beginning and the end of activities for each machine.

In [6]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)
clear_output(wait=True)
for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Job 1 completed at 309.0s
Job 2 completed at 403.0s
Job 3 completed at 92.0s
Job 4 completed at 902.0s
Job 5 completed at 494.0s
Job 6 completed at 30.0s
Job 7 completed at 777.0s
Job 8 completed at 815.0s
Job 9 completed at 223.0s
Job 10 completed at 743.0s
Job 11 completed at 914.0s
Job 12 completed at 305.0s
Job 13 completed at 103.0s
Job 14 completed at 859.0s
Job 15 completed at 702.0s
Job 16 completed at 634.0s
Job 17 completed at 794.0s
Job 18 completed at 695.0s
Job 19 completed at 31.0s
Job 20 completed at 340.0s
Job 21 completed at 832.0s
Job 22 completed at 942.0s
Job 23 completed at 540.0s
Job 24 completed at 619.0s
Job 25 completed at 393.0s
Job 26 completed at 729.0s
Job 27 completed at 887.0s
Job 28 completed at 235.0s
Job 29 completed at 332.0s
Job 30 completed at 656.0s
Job 31 completed at 238.0s
Job 32 completed at 23.0s
Job 33 completed at 113.0s
Job 34 completed at 151.0s
Job 35 completed at 747.0s
Job 36 completed at 488.0s
Job 37 completed at 512.0s
Job 38 complet

# Mathematical Programming Algorithm (MPA)

The paper cites a decomposition algorithm (Branch-and-check) based on the previous formulation.

It is an exact formulation based on a master algorithm and a sequence algorithm:
- The master algorithm is responsible of assigning jobs to machines
- The sequence problem is responsible of finding the optimal sequence of jobs for each machine

In [7]:
s = MPA(execution_times = P_dict, setup_times = S_dict, t_max=10800) #  max_time = 1800
decision_variables,maximum_makespan = s.solve(options=options)
clear_output(wait=True)
for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

Job 0 scheduled in sequence of job 32 in machine 1
Job 2 scheduled in sequence of job 47 in machine 1
Job 3 scheduled in sequence of job 13 in machine 1
Job 6 scheduled in sequence of job 3 in machine 1
Job 7 scheduled in sequence of job 17 in machine 1
Job 10 scheduled in sequence of job 7 in machine 1
Job 12 scheduled in sequence of job 40 in machine 1
Job 13 scheduled in sequence of job 44 in machine 1
Job 17 scheduled in sequence of job 48 in machine 1
Job 18 scheduled in sequence of job 10 in machine 1
Job 21 scheduled in sequence of job 27 in machine 1
Job 22 scheduled in sequence of job 46 in machine 1
Job 23 scheduled in sequence of job 24 in machine 1
Job 24 scheduled in sequence of job 30 in machine 1
Job 25 scheduled in sequence of job 2 in machine 1
Job 27 scheduled in sequence of job 22 in machine 1
Job 30 scheduled in sequence of job 18 in machine 1
Job 31 scheduled in sequence of job 50 in machine 1
Job 32 scheduled in sequence of job 6 in machine 1
Job 36 scheduled in s

# Fix-and-Optimize approach

In [8]:
greedy = Greedy(execution_times=P_dict, setup_times=S_dict)
solution = greedy.solve()

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

Job 0 scheduled in sequence of job 30 in machine 1
Job 2 scheduled in sequence of job 11 in machine 1
Job 3 scheduled in sequence of job 0 in machine 1
Job 4 scheduled in sequence of job 6 in machine 1
Job 5 scheduled in sequence of job 4 in machine 1
Job 6 scheduled in sequence of job 3 in machine 1
Job 8 scheduled in sequence of job 12 in machine 1
Job 9 scheduled in sequence of job 48 in machine 1
Job 11 scheduled in sequence of job 23 in machine 1
Job 12 scheduled in sequence of job 26 in machine 1
Job 15 scheduled in sequence of job 44 in machine 1
Job 16 scheduled in sequence of job 2 in machine 1
Job 22 scheduled in sequence of job 29 in machine 1
Job 23 scheduled in sequence of job 39 in machine 1
Job 26 scheduled in sequence of job 16 in machine 1
Job 27 scheduled in sequence of job 22 in machine 1
Job 29 scheduled in sequence of job 15 in machine 1
Job 30 scheduled in sequence of job 42 in machine 1
Job 34 scheduled in sequence of job 9 in machine 1
Job 39 scheduled in sequen

In [9]:
m = FixOpt(initial_solution=solution, setup_times=S_dict,execution_times=P_dict, N=N, M=M, N0=N0, 
           subproblem_size_adjust_rate=0.1, t_max = 60, subproblem_runtime_limit=10, subproblem_size=5)

solution, makespan = m.solve()
clear_output(wait=True)
for i in M:
    for j in N0:
        for k in N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled after job {k} in machine {i}')
print(f'The makespan has been minimized to {makespan}s')

Job 0 scheduled after job 32 in machine 1
Job 2 scheduled after job 47 in machine 1
Job 3 scheduled after job 13 in machine 1
Job 6 scheduled after job 3 in machine 1
Job 7 scheduled after job 17 in machine 1
Job 10 scheduled after job 7 in machine 1
Job 12 scheduled after job 40 in machine 1
Job 13 scheduled after job 44 in machine 1
Job 17 scheduled after job 48 in machine 1
Job 18 scheduled after job 10 in machine 1
Job 21 scheduled after job 27 in machine 1
Job 22 scheduled after job 46 in machine 1
Job 23 scheduled after job 24 in machine 1
Job 24 scheduled after job 30 in machine 1
Job 25 scheduled after job 2 in machine 1
Job 27 scheduled after job 22 in machine 1
Job 30 scheduled after job 18 in machine 1
Job 31 scheduled after job 50 in machine 1
Job 32 scheduled after job 6 in machine 1
Job 36 scheduled after job 37 in machine 1
Job 37 scheduled after job 23 in machine 1
Job 40 scheduled after job 25 in machine 1
Job 43 scheduled after job 31 in machine 1
Job 44 scheduled aft